In [1]:
import os
import re
import json
import shutil
import geemap
import numpy as np
import pandas as pd
from ipywidgets import HTML
from ipyleaflet import Marker, MarkerCluster,Popup

In [2]:
df = pd.read_csv('Wind_farm_lon_lat_geometry_batch1.csv',header = None)

In [3]:
lonlat = df.drop_duplicates(subset=[0])

In [4]:
raw_db = pd.read_excel(
    "Global-Wind-Power-Tracker-May-2023.xlsx", "Data", usecols="B,C,G,H,I,J,P,Q,R"
)
turbine_type = ["offshore floating", "offshore hard mount", "offshore mount unknown"]
offshore_db = raw_db[raw_db["Installation Type"].isin(turbine_type)]
active_offshore_db = offshore_db[offshore_db["Status"] == "operating"]
active_offshore_db_accurate = active_offshore_db[
    active_offshore_db["Location accuracy"] == "exact"
]

In [5]:
active_offshore_db_accurate

,Country,Project Name,Capacity (MW),Installation Type,Status,Start year,Latitude,Longitude,Location accuracy
481,Japan,Kamis Offshore wind farm,14.0,offshore hard mount,operating,2010.0,35.8905,140.7297,exact
482,Japan,Kamis Offshore wind farm,16.0,offshore hard mount,operating,2013.0,35.9025,140.7209,exact
585,Japan,Sakata North Port (Nearshore) Offshore wind farm,16.0,offshore hard mount,operating,2014.0,38.9689,139.8231,exact
1212,Taiwan,Formosa 1 offshore wind farm,120.0,offshore hard mount,operating,2019.0,24.7092,120.8151,exact
1351,Vietnam,Bạc Liêu Offshore wind farm,16.0,offshore hard mount,operating,2013.0,9.2367,105.8223,exact
...,...,...,...,...,...,...,...,...,...
20920,United Kingdom,Walney wind farm,184.0,offshore hard mount,operating,2012.0,54.0807,-3.6090,exact
20940,United Kingdom,West Of Duddon Sands wind farm,389.0,offshore hard mount,operating,2014.0,53.9851,-3.4620,exact
20943,United Kingdom,Westermost Rough A wind farm,210.0,offshore hard mount,operating,2015.0,53.0977,0.5615,exact
21657,United States,Block Island wind farm,30.0,offshore hard mount,operating,2016.0,41.1199,-71.5139,exact


In [6]:
lonlat.insert(8,'Country','Na')

In [7]:
for index, row in lonlat.iterrows():
    # print(active_offshore_db_accurate.iloc[lonlat[0][index]]['Country'])
    lonlat['Country'][index] = active_offshore_db_accurate.iloc[lonlat[0][index]]['Country']

/var/tmp/ipykernel_3326370/3396466594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lonlat['Country'][index] = active_offshore_db_accurate.iloc[lonlat[0][index]]['Country']


In [8]:
lonlat[lonlat[0] == 174]

,0,1,2,3,4,5,6,7,Country
232,174,-3.65,53.34,0.06,01/03/2023,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[-3.71, 5...","[{'type': 'Polygon', 'coordinates': [[[-3.6390...",United Kingdom


In [9]:
countries = set(active_offshore_db_accurate["Country"])

In [10]:
countries

{'Belgium',
 'China',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Italy',
 'Japan',
 'Netherlands',
 'Sweden',
 'Taiwan',
 'United Kingdom',
 'United States',
 'Vietnam'}

In [11]:
active_offshore_db_accurate['Country'][481]

'Japan'

In [12]:
from google.auth import compute_engine
import ee
credentials = compute_engine.Credentials(scopes=['https://www.googleapis.com/auth/earthengine'])
ee.Initialize(credentials)

In [13]:
lonlat[6][0]

'{"type": "Polygon", "coordinates": [[[121.03330000000001, 33.408300000000004], [121.03330000000001, 33.2083], [121.2333, 33.2083], [121.2333, 33.408300000000004]]], "evenOdd": true}'

In [14]:
square = ee.Geometry(json.loads(lonlat[6][0]))

In [15]:
len(lonlat)

151

In [16]:
lonlat[lonlat['Country'] == 'China']

,0,1,2,3,4,5,6,7,Country
0,38,121.1333,33.3083,0.100,01/01/2022,31/03/2022,"{""type"": ""Polygon"", ""coordinates"": [[[121.0333...","[{'type': 'Polygon', 'coordinates': [[[121.233...",China
2,40,121.3750,33.4600,0.075,01/03/2022,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.3, 3...","[{'type': 'Polygon', 'coordinates': [[[121.435...",China
4,42,121.5927,33.3225,0.080,01/10/2022,30/01/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.5127...","[{'type': 'Polygon', 'coordinates': [[[121.669...",China
6,43,121.2700,32.9348,0.080,01/03/2023,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.19, ...","[{'type': 'Polygon', 'coordinates': [[[121.195...",China
8,45,121.1378,32.6845,0.075,01/10/2022,30/01/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.0628...","[{'type': 'Polygon', 'coordinates': [[[121.063...",China
10,46,122.0501,32.1600,0.050,01/03/2023,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[122.0001...","[{'type': 'Polygon', 'coordinates': [[[122.070...",China
12,47,122.1828,32.0800,0.080,01/10/2022,30/01/2023,"{""type"": ""Polygon"", ""coordinates"": [[[122.1028...","[{'type': 'Polygon', 'coordinates': [[[122.227...",China
14,48,121.6917,32.3739,0.080,01/03/2023,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.6117...","[{'type': 'Polygon', 'coordinates': [[[121.616...",China
16,51,120.9994,32.6038,0.060,01/03/2023,30/06/2023,"{""type"": ""Polygon"", ""coordinates"": [[[120.9393...","[{'type': 'Polygon', 'coordinates': [[[120.994...",China
18,52,121.2583,32.6583,0.100,01/10/2022,30/01/2023,"{""type"": ""Polygon"", ""coordinates"": [[[121.1583...","[{'type': 'Polygon', 'coordinates': [[[121.358...",China


In [17]:
countries

{'Belgium',
 'China',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Italy',
 'Japan',
 'Netherlands',
 'Sweden',
 'Taiwan',
 'United Kingdom',
 'United States',
 'Vietnam'}

In [19]:
map1 = geemap.Map()
country = 'China'
subset = lonlat[lonlat['Country'] == country]
# subset = lonlat
for index, row in subset.iterrows():
    marker = Marker(location = (subset[2].loc[index],subset[1].loc[index]), name = str(subset[0].loc[index]))
    message = HTML()
    message.value = str(subset[0].loc[index])
    popup = Popup()
    marker.popup = message
    map1.add(marker)
    square = ee.Geometry(json.loads(subset[6].loc[index]))
    map1.addLayer(square)
    
map1

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [1]:
# around 105
train_id = [0,14, 9, 1, 32, 30, 33, 23, 24, 35, 15, 19, 16, 17, 18, 11, 12 ,13,76 ,83 ,78 , 47, 46, 54, 48, 57, 53, 64, 45, 52, 55, 56, 58, 59, 60, 61, 62, 37, 42, 67, 65, 66, 71,72, 69
           , 88, 84, 85, 87,100, 101, 98, 95, 89, 99, 96, 91, 108, 131, 107, 121, 124, 122, 119, 118,128, 123, 105, 110, 111, 130, 114, 120, 104, 117, 135, 134, 133, 142, 140, 3,
             4, 163, 154, 167, 150, 176, 149, 166, 162, 180, 160, 171, 178, 179, 146, 165, 143, 161, 181, 170, 144, 147, 169, 157]
#around 30
val_id = [22, 34, 10, 80, 82, 63, 43, 40, 38, 68, 73 ,74, 86,97, 90, 93, 92, 115, 113, 112, 127, 126, 139, 141, 8, 173, 152, 177, 184, 145, 175]
#around 15
test_id = [79, 81, 51,70, 94, 102, 116, 109, 132, 137, 6, 158, 174, 186, 185]

In [2]:
len(train_id)

105

186, 185 to test set
14,9 to train

In [57]:
val_id.count(38)

1

In [58]:
print(0.7*len(subset))
print(0.2*len(subset))
print(0.1*len(subset))

0.7
0.2
0.1


In [59]:
print(len(train_id),len(val_id),len(test_id))

105 31 15


In [60]:
len(train_id)+len(val_id)+len(test_id)

151

## Put the matrix into the proper folder

In [61]:
def copy_in_folder(original_dir, destination_dir, file_name):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    source_path = os.path.join(original_dir, file_name)
    destination_path = os.path.join(destination_dir, file_name)
    if not os.path.exists(destination_path):
        shutil.copy2(source_path, destination_path)

In [62]:
# destination = os.path.join(path,'val')

# os.path.exists(destination)
# copy_in_folder(path, destination, 'bands_0_field_38_period_01-01-2022_to_31-03-2022.npy')

In [63]:
def put_files_in_folder(file_path, train_id, val_id, test_id):
    #get the list of unclassified matrix
    unclassified_list = sorted(os.listdir(file_path))
    df = pd.DataFrame(unclassified_list,columns  = ['file name'])
    # print(df)
    df = df[df['file name'].str.endswith('.npy')]
    # print(df)
    def id_extraction(x):
        pattern = 'd_(\d+)'
        numb = int(re.findall(pattern, x)[0])
        return numb
    df['id'] = df['file name'].apply(id_extraction)
    # print(df)
    for index, row in df.iterrows():
        if train_id.count(row['id']):
            destination = os.path.join(path,'train')
            copy_in_folder(path, destination, row['file name'])
        elif val_id.count(row['id']):
            destination = os.path.join(path,'val')
            copy_in_folder(path, destination, row['file name'])
        elif test_id.count(row['id']):
            destination = os.path.join(path,'test')
            copy_in_folder(path, destination, row['file name'])

In [27]:
path = os.path.realpath(
            os.path.join(
                os.path.curdir,
                "..",
                "..",
                "data",
                "processed",
                "bands"))
put_files_in_folder(path, train_id, val_id, test_id)

In [28]:
path = os.path.realpath(
            os.path.join(
                os.path.curdir,
                "..",
                "..",
                "data",
                "processed",
                "masks"))
put_files_in_folder(path, train_id, val_id, test_id)

## Save in a bucket

In [3]:
bucket_name = "wind-turbine-project-thibaud"
my_prefix = "image & ground truth/processed/"
# filename = "osm_wt_g.parquet"

In [4]:
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

In [5]:
def upload_blob(bucket_name, source_directory, destination_blob_name):
    """Uploads a directory to the bucket."""
    
    storage_client = storage.Client()
    
    bucket = storage_client.bucket(bucket_name)
    
    pattern = '\w+\/(?:(?:train)|(?:val)|(?:test))'
    
    for dirpath, dirs, filenames in os.walk(source_directory):
        
        if (dirs == [] or dirs == ['.ipynb_checkpoints'])and dirpath[-17:]!= 'ipynb_checkpoints':
            
            f = re.findall(pattern,dirpath)[0]
            
            for file in filenames:
                blob = bucket.blob(os.path.join(destination_blob_name, f, file))

                blob.upload_from_filename(os.path.join(destination_blob_name, dirpath, file))

In [95]:
source_directory = path = os.path.realpath(
            os.path.join(
                os.path.curdir,
                "..",
                "..",
                "data",
                "processed",
                "bands"))

destination_blob_name = "image & ground truth/processed/"

bucket_name = "wind-turbine-project-thibaud"

upload_blob(bucket_name, source_directory, destination_blob_name)

In [6]:
source_directory = path = os.path.realpath(
            os.path.join(
                os.path.curdir,
                "..",
                "..",
                "data",
                "processed",
                "masks"))

destination_blob_name = "image & ground truth/processed/"

bucket_name = "wind-turbine-project-thibaud"

upload_blob(bucket_name, source_directory, destination_blob_name)